<div style="font-size: 200%; font-weight: bold; color: maroon;">Spark $\pi$</span></div>
<!-- v 2.0 -- Paulo Villegas, 2021 -->


Vamos a hacer ahora un pequeño ejemplo de Spark con Scala 

El programa calcula $\pi$ con un margen de error mediante la fórmula del área del círculo unidad (que es $A = \pi  r^2 = \pi$). Para calcularlo, muestreamos el cuadrado unidad repetidamente (generando números aleatorios entre 0 y 1) y contamos la proporción de puntos que caen dentro del círculo unidad; el resultado es una aproximación a un área que es $1/4$ del círculo unidad, luego al final multiplicamos el resultado por 4. 

Para más detalles, ver [la página en wikipedia](https://en.wikipedia.org/wiki/Approximations_of_%CF%80#Summing_a_circle.27s_area) o [esta otra explicación](https://www.cantorsparadise.com/calculating-the-value-of-pi-using-random-numbers-a-monte-carlo-simulation-d4b80dc12bdf)

# Programa original
Este es el código original del ejemplo de [SparkPi](https://github.com/apache/spark/blob/master/examples/src/main/scala/org/apache/spark/examples/SparkPi.scala) disponible en la sección de Scala de los ejemplos de Spark.

```Scala
import scala.math.random
import org.apache.spark._

/** Computes an approximation to pi */
object SparkPi {
  def main(args: Array[String]): Unit = {
    val spark = SparkSession
      .builder
      .appName("Spark Pi")
      .getOrCreate()
    val slices = if (args.length > 0) args(0).toInt else 2
    val n = math.min(100000L * slices, Int.MaxValue).toInt // avoid overflow
    val count = spark.sparkContext.parallelize(1 until n, slices).map { i =>
      val x = random * 2 - 1
      val y = random * 2 - 1
      if (x*x + y*y <= 1) 1 else 0
    }.reduce(_ + _)
    println(s"Pi is roughly ${4.0 * count / (n - 1)}")
    spark.stop()
  }
}
```

<div class="alert alert-info">Nota: este código es ligeramente distinto a lo explicado anteriormente. El ejemplo calcula el <strong>área completa</strong> del círculo unidad:
    <ul><li>Muestrea las posiciones en todo el círculo, es decir, en [-1, 1].
        <li>El área completa de ese círculo es precisamente $A_1 = \pi\cdot {1}^{2} = \pi$, 
        <li>mientras que el área del cuadrado en el que está inscrito el círculo es $A_c = 2^2 = 4$. 
        <li>El ratio entre ambas áreas es, por tanto $ A_1 / A_c = \pi / 4 $. 
        <li>Luego si calculamos el ratio entre los puntos dentro del círculo unidad y fuera, y multiplicamos por 4, volvemos a obtener $\pi$ 
    </ul></div>

Para empezar a trabajar antes de nada necesitamos cargar Spark en el kernel de Scala

# Carga de Spark

En el kernel de Scala que estamos usando ([almond](https://almond.sh/)) Spark no viene precargado por defecto, como en el de PySpark. 
* Es posible [importar Spark](https://almond.sh/docs/usage-spark) usando funcionalidad especial de _almond_ que descarga automáticamente los paquetes de Spark de red y los instala.
* Pero como nosotros tenemos _ya_ una versión instalada localmente en la máquina virtual, lo que vamos a hacer es añadir sus paquetes (jars) al kernel de Scala, para que los cargue

In [ ]:
// Load the Spark Jars into the Scala interpreter
def loadSpark(path: Option[String] = None ) = {
    // Directory where to look for jars to load
    val jardir = path.getOrElse("/opt/spark/current") + "/jars"
    // Get all jars in the directory
    val jarlist = java.nio.file.Files.list(java.nio.file.Paths.get(jardir)).toArray.map(_.toString);
    // Convert into a sequence of os.Path objects that can be loaded 
    val seq = jarlist.map( fname => ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(fname)) );
    // Load all jars into the interpreter
    interp.load.cp(seq);
}

loadSpark(sys.env.get("SPARK_HOME"));

In [ ]:
// Reduce a bit the logs from Spark to avoid too much information in the notebook
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org.apache").setLevel(Level.WARN)

Ahora podemos crear una sesión local de Spark 

In [ ]:
import org.apache.spark.sql._

val spark = SparkSession.builder().master("local[*]").appName("Spark Pi").getOrCreate()

# Ejecución

## como programa completo

In [ ]:
import scala.math.random
import org.apache.spark._

/** Computes an approximation to pi */
object SparkPi {
  def main(args: Array[String]) {
    val slices = if (args.length > 0) args(0).toInt else 2
    val n = 100000 * slices
    val count = spark.sparkContext.parallelize(1 to n, slices).map { i =>
      val x = random * 2 - 1
      val y = random * 2 - 1
      if (x*x + y*y < 1) 1 else 0
    }.reduce(_ + _)

    println("Pi is roughly " + 4.0 * count / n)
  }
}

Una vez definido el programa, podemos ejecutarlo llamando a su punto de entrada

In [ ]:
SparkPi.main( Array("4") )

## como notebook
Ahora vamos a hacer lo mismo, pero en un formato más _notebook_, dividiendo el programa en celdas que ejecutaremos en secuencia. Usaremos también el algoritmo explicado en primer lugar (cálculo usando solo el cuadrante positivo)

In [ ]:
// Imports we need
import scala.math.random
import org.apache.spark._

In [ ]:
// Define how many threads we want
val slices = 16
// We will sample 100K points per thread
val n = math.min(100000L * slices, Int.MaxValue).toInt // avoid overflow

In [ ]:
// the program
val count = spark.sparkContext.parallelize(1 to n, slices).map { i =>
      val x = random
      val y = random
      if (x*x + y*y <= 1) 1 else 0
    }.reduce(_ + _)

In [ ]:
// The result
println("Pi is roughly " + 4.0 * count / n)